In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Non-std libs
import matplotlib.pyplot as plt

## Local libs
from scn_rrd import rrd_utils, rrd_meta_utils

### Count users by counting UEs

In [ ]:
class At_eNB:
    def __init__(self):
        self.locations = [
            'FCS',
            'sps-franklin',
        ]
        
        self.meta = rrd_meta_utils.read_meta()
        self.meta = self.meta[
            ( self.meta['location'].isin(self.locations) ) &
            ( self.meta['device_group_name'] == 'eNB' )
        ]
        self.deviceHostnames = self.meta['hostname'].unique()
    
    def read_rrds(self):
        self.deviceHostname_to_df = rrd_utils.read_rrds(
            self.deviceHostnames,
            rrd_meta_utils.ENB_DEVICE_RRD_FILENAMES.clients,
            '-6month'
        )

In [ ]:
at_enb = At_eNB()

In [ ]:
at_enb.read_rrds()
# Right now, BaicellsNova436Q devices do not have wireless-sensor-clients-rts-1.rrd.
# If we see stderr here, this is expected.

### Count users by DHCP

In [ ]:
class At_DHCP:
    def __init__(self):
        self.locations = [
            'nickelsville-cd',
            'northlake',
        ]
        
        self.meta = rrd_meta_utils.read_meta()
        self.meta = self.meta[
            ( self.meta['location'].isin(self.locations) ) &
            ( self.meta['hostname'].str.contains('Gateway', case=False) ) &
            ( self.meta['hostname'].str.contains('hEXS', case=False) )
        ]
        self.deviceHostnames = self.meta['hostname'].unique()
    
    def read_rrds(self):
        self.deviceHostname_to_df = rrd_utils.read_rrds(
            self.deviceHostnames,
            rrd_meta_utils.MIKROTIK_DEVICE_RRD_FILENAMES.clients,
            '-6month'
        )

In [ ]:
at_dhcp = At_DHCP()

In [ ]:
at_dhcp.read_rrds()

### Plot

In [ ]:
(fig, ax) = plt.subplots()
for (deviceHostname, df) in at_enb.deviceHostname_to_df.items():
    ax.plot(df['time'], df['sensor'], label=deviceHostname)
for (deviceHostname, df) in at_dhcp.deviceHostname_to_df.items():
    ax.plot(df['time'], df['leases'], label=deviceHostname)
ax.set_title('Count of users')
ax.legend()